In [141]:
import kashgari
from kashgari.embeddings import NumericFeaturesEmbedding, BareEmbedding, StackedEmbedding, BERTEmbedding

import logging
logging.basicConfig(level='DEBUG')

In [169]:
#BERT MODEL 

bert_model_path= '/Users/ishitagupta/bert/uncased_L-12_H-768_A-12'
bert_embedding = BERTEmbedding(bert_model_path,
                               task=kashgari.LABELING,
                               sequence_length=SEQUENCE_LEN)

tokenizer = bert_embedding.tokenizer


In [250]:
text = [['[CLS]', 'nl', '##p', '[SEP]'], ['[CLS]','projects','[SEP]'], ['[CLS]','project', '[SEP]'], ['[CLS]', 'name', '[SEP]'], ['[CLS]', ':', '[SEP]']]
start_of_p = [1, 2, 1, 2, 2]
bold = [1, 1, 1, 1, 2]
center = [1, 1, 2, 2, 2]
label = ['B-Category', 'I-Category', 'B-ProjectName', 'I-ProjectName', 'B-ProjectName']


In [251]:
text_list = text
start_of_p_list = [start_of_p] 
bold_list = [bold] 
center_list = [center] 
label_list = [label] 

In [214]:
SEQUENCE_LEN = 100

In [215]:
# You can use Word Embedding or BERT Embedding for your text embedding
text_embedding = BareEmbedding(task=kashgari.LABELING, sequence_length=SEQUENCE_LEN)
start_of_p_embedding = NumericFeaturesEmbedding(feature_count=2,
                                                feature_name='start_of_p',
                                                sequence_length=SEQUENCE_LEN)

bold_embedding = NumericFeaturesEmbedding(feature_count=2,
                                                feature_name='bold',
                                                sequence_length=SEQUENCE_LEN)

center_embedding = NumericFeaturesEmbedding(feature_count=2,
                                                feature_name='center',
                                                sequence_length=SEQUENCE_LEN)


DEBUG:root:need to build after build_word2idx


In [216]:
# first attribute, must be the text embedding
stack_embedding = StackedEmbedding([
    bert_embedding,
    start_of_p_embedding,
    bold_embedding,
    center_embedding
])

[<tf.Tensor 'Input-Token_5:0' shape=(?, 100) dtype=float32>, <tf.Tensor 'Input-Segment_5:0' shape=(?, 100) dtype=float32>]
[<tf.Tensor 'Input-Token_5:0' shape=(?, 100) dtype=float32>, <tf.Tensor 'Input-Segment_5:0' shape=(?, 100) dtype=float32>]
Tensor("input_start_of_p_9:0", shape=(?, 100), dtype=float32)
[<tf.Tensor 'input_start_of_p_9:0' shape=(?, 100) dtype=float32>]
Tensor("input_bold_9:0", shape=(?, 100), dtype=float32)
[<tf.Tensor 'input_bold_9:0' shape=(?, 100) dtype=float32>]
Tensor("input_center_9:0", shape=(?, 100), dtype=float32)
[<tf.Tensor 'input_center_9:0' shape=(?, 100) dtype=float32>]


In [252]:
x = (text_list, start_of_p_list, bold_list, center_list)
y = label_list
stack_embedding.analyze_corpus(x, y)

In [257]:
print(len(x[0]))
print(len(y[0]))

5
5


In [254]:

# Now we can embed using this stacked embedding layer
print(stack_embedding.embed(x))

IndexError: index 1 is out of bounds for axis 0 with size 1

In [255]:
# We can build any labeling model with this embedding

from kashgari.tasks.labeling import BiLSTM_Model
model = BiLSTM_Model(embedding=stack_embedding)
model.fit(x, y, x, y)



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 100)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 100)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 100, 768), ( 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 100, 768)     1536        Input-Segment[0][0]              
__________________________________________________________________________________________________
Embedding-

Epoch 1/5


ValueError: All input arrays (x) should have the same number of samples. Got array shapes: [(4, 100), (4, 100), (1, 100), (1, 100), (1, 100)]

In [ ]:
print(model.predict(x))


In [ ]:
print(model.predict_entities(x))